In [13]:
import cv2
import os, sys
import numpy as np
from keras.models import  model_from_json
from keras.preprocessing import image


In [14]:
loaded_model = model_from_json(open('emotion.json','r').read())
loaded_model.load_weights('emotion_weights.h5')

In [15]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [16]:
cap = cv2.VideoCapture(0)

In [17]:
while True:
    ret, test_img = cap.read()
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    faces_detected = face_cascade.detectMultiScale(gray_img, 1.3, 5)
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y), (x+w,y+h),(255,0,0),1 )
        roi_gray = gray_img[y:y+w, x:x+h]
        roi_gray = cv2.resize(roi_gray, (48,48))
        image_pixels = image.img_to_array(roi_gray)
        image_pixels = np.expand_dims(image_pixels, axis=0)
        image_pixels /= 255
        pred = loaded_model.predict(image_pixels)
        max_index = np.argmax(pred[0])
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        pred_emo = emotions[max_index]
        cv2.putText(test_img, pred_emo, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    resized_img = cv2.resize(test_img, (1000,700))
    cv2.imshow('frame', resized_img)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    